# Download data from opensecrets.org website

As the number of downloads is limited, I commented the lines that do the download.
The un-commented lines shows the data from the saved files.

In [1]:
import requests
import json
import pandas as pd

In [54]:
###downloads the legislators main infos

# r = requests.get("https://www.opensecrets.org/api/?method=getLegislators&id=NJ&apikey=925fb8825456ce94554138f1945e41de&output=json")

# j=r.json()
# final=[]
# for data in j["response"]["legislator"]:
#     final.append(data['@attributes'])
# print(str(final)[:1000])
# file=open('candidates_list.json', 'w')
# file.write(json.dumps(final))
# file.close()

In [55]:
#loading 1 candidate
file=open('candidates_list.json', 'r')
j=json.loads(file.readline())
file.close()
j[0]

{'bioguide_id': 'N000188',
 'birthdate': '1958-12-13',
 'cid': 'N00036154',
 'comments': '',
 'congress_office': '1531 Longworth House Office Building',
 'exit_code': '0',
 'facebook_id': 'DonaldNorcrossNJ',
 'fax': '202-225-6583',
 'feccandid': 'H4NJ01084',
 'first_elected': '2014',
 'firstlast': 'Don Norcross',
 'gender': 'M',
 'lastname': 'NORCROSS',
 'office': 'NJ01',
 'party': 'D',
 'phone': '202-225-6501',
 'twitter_id': 'DonaldNorcross',
 'votesmart_id': '',
 'webform': 'http://norcross.house.gov/contact',
 'website': 'https://norcross.house.gov',
 'youtube_url': ''}

In [57]:
#download 1 method, for 1 candidate, for all years
def downloadType(method, val1, val2, cid='N00007360', apikey="925fb8825456ce94554138f1945e41de"):
    final=[]
    years=[2014, 2016, 2018]
    
    for year in years:
        param={'output':'json', 'method':method, 'cid':cid, 'cycle':year, \
               'apikey':apikey}
        r = requests.get('https://www.opensecrets.org/api/', param)
        #print(r.url)
        try:
            j=r.json()
            #transform the json
            for data in j["response"][val1][val2]:
                d=data["@attributes"]
                d["year"]=year
                d["cid"]=cid
                final.append(d)
        except:
            #"Resource not found" case
            pass
    #return value
    return final



#print(downloadType1("candContrib", "contributors", "contributor", "N00036944"))
#print(downloadType1("candIndustry", "industries", "industry"))
#print(downloadType1("candSector", "sectors", "sector"))

In [58]:
#download data for 1 method, all candidates, and all years
def downloadMethod(method, val1, val2, output_file=None, candidates_file='candidates_list.json', \
                   apikey="925fb8825456ce94554138f1945e41de"):
    if output_file is None:
        output_file=method+'.json'
    
    cand_file=open('candidates_list.json', 'r')
    candidates=json.loads(cand_file.readline())
    cand_file.close()

    out_file=open(output_file, 'w')
    
    for candidate in candidates:
        print(candidate['cid'])
        data=downloadType(method, val1, val2, candidate['cid'])
        out_file.write(json.dumps(data))
        out_file.write('\n')
    
    out_file.close()

In [60]:
#downloadMethod("candContrib", "contributors", "contributor")

In [61]:
#downloadMethod("candIndustry", "industries", "industry")

In [62]:
#downloadMethod("candSector", "sectors", "sector")

In [63]:
#function that open the file and create a "good" dataset
def createDataFrame(file):
    ##lists of 50 data are loaded and then merged
    #create list of dataframes and open file
    dataframes=[]
    file=open(file, 'r')
    
    #counter is important for the index
    c=0
    while True:
        #each line is 50 data (as limit=50 when downloading)
        txt=file.readline()
        #possibly stop
        if txt=='':
            break

        #load the data with right index
        j=json.loads(txt)
        l=len(j)
        dataframes.append(pd.DataFrame(j, index=list(range(c, c+l))))
        
        c+=l
    #merge all datasets and return result
    return pd.concat(dataframes)


In [64]:
dfIndustry=createDataFrame("candIndustry.json")
dfIndustry

,cid,indivs,industry_code,industry_name,pacs,total,year
0,N00036154,2000,P01,Building Trade Unions,170499,172499,2014
1,N00036154,123706,K01,Lawyers/Law Firms,25100,148806,2014
2,N00036154,104306,F10,Real Estate,27000,131306,2014
3,N00036154,59750,C04,Construction Services,16800,76550,2014
4,N00036154,1000,Q03,Leadership PACs,66000,67000,2014
5,N00036154,0,P04,Public Sector Unions,66000,66000,2014
6,N00036154,1150,P02,Industrial Unions,62000,63150,2014
7,N00036154,58000,F13,Misc Finance,0,58000,2014
8,N00036154,13350,H01,Health Professionals,36000,49350,2014
9,N00036154,0,P03,Transportation Unions,42500,42500,2014


In [65]:
dfSector=createDataFrame("candSector.json")
dfSector

,cid,indivs,pacs,sector_name,sectorid,total,year
0,N00036154,28516,22500,Agribusiness,A,51016,2014
1,N00036154,16300,13000,Communic/Electronics,B,29300,2014
2,N00036154,104300,29900,Construction,C,134200,2014
3,N00036154,2600,9000,Defense,D,11600,2014
4,N00036154,21700,35000,Energy/Nat Resource,E,56700,2014
5,N00036154,199422,56100,Finance/Insur/RealEst,F,255522,2014
6,N00036154,68750,68850,Health,H,137600,2014
7,N00036154,151061,29140,Lawyers & Lobbyists,K,180201,2014
8,N00036154,18000,17500,Transportation,M,35500,2014
9,N00036154,39381,18500,Misc Business,N,57881,2014


In [66]:
dfContrib=createDataFrame("candContrib.json")
dfContrib

,cid,indivs,org_name,pacs,total,year
0,N00036154,30400,"Conner, Strong & Buckelew",0,30400,2014
1,N00036154,1000,Plumbers/Pipefitters Union,25000,26000,2014
2,N00036154,21800,Charter School Management,0,21800,2014
3,N00036154,1150,Intl Brotherhood of Electrical Workers,20000,21150,2014
4,N00036154,1000,Carpenters & Joiners Union,20000,21000,2014
5,N00036154,20950,Cooper University Hospital,0,20950,2014
6,N00036154,0,American Federation of Teachers,20000,20000,2014
7,N00036154,0,Ironworkers Union,20000,20000,2014
8,N00036154,0,Laborers Union,20000,20000,2014
9,N00036154,0,Painters & Allied Trades Union,20000,20000,2014


In [42]:
file=open('candidates_list.json', 'r')

j=json.loads(file.readline())
print(len(j))
dfCandidates=pd.DataFrame(j)
dfCandidates

14


,bioguide_id,birthdate,cid,comments,congress_office,exit_code,facebook_id,fax,feccandid,first_elected,...,gender,lastname,office,party,phone,twitter_id,votesmart_id,webform,website,youtube_url
0,N000188,1958-12-13,N00036154,,1531 Longworth House Office Building,0,DonaldNorcrossNJ,202-225-6583,H4NJ01084,2014,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
1,L000554,1946-05-12,N00000851,Retiring at end of 115th,2427 Rayburn House Office Building,4,FrankLoBiondo,202-225-3318,H2NJ02037,1994,...,M,LOBIONDO,NJ02,R,202-225-6572,RepLoBiondo,21890,https://lobiondo.house.gov/contact-me/email-me,http://lobiondo.house.gov,https://youtube.com/USRepFrankLoBiondo
2,M001193,1960-10-16,N00036155,,506 Cannon House Office Building,0,CongressmanTomMacArthur,202-225-0778,H4NJ03130,2014,...,M,MACARTHUR,NJ03,R,202-225-4765,RepTomMacArthur,,,https://macarthur.house.gov,
3,S000522,1953-03-04,N00009816,,2373 Rayburn House Office Building,0,RepChrisSmith,202-225-7768,H8NJ04014,1980,...,M,SMITH,NJ04,R,202-225-3765,RepChrisSmith,26952,https://chrissmith.house.gov/contact/write.htm,http://chrissmith.house.gov,https://youtube.com/USRepChrisSmith
4,G000583,1975-03-08,N00036944,,213 Cannon House Office Building,0,RepJoshG,,,2016,...,M,GOTTHEIMER,NJ05,D,202-225-4465,RepJoshG,,,https://gottheimer.house.gov,
5,P000034,1951-10-30,N00000781,,237 Cannon House Office Building,0,repfrankpallone,202-225-9665,H8NJ03073,1988,...,M,PALLONE,NJ06,D,202-225-4671,FrankPallone,26951,https://pallone.house.gov/contact-me/email-me,https://pallone.house.gov,https://youtube.com/repfrankpallone
6,L000567,1952-06-25,N00000898,,2352 Rayburn House Office Building,0,CongressmanLance,202-225-9460,H6NJ12136,2008,...,M,LANCE,NJ07,R,202-225-5361,RepLanceNJ7,4443,https://lance.house.gov/contact-me1,http://lance.house.gov,https://youtube.com/CongressmanLance
7,S001165,1951-01-26,N00027523,,2342 Rayburn House Office Building,0,81058818750,202-226-0792,H6NJ13191,2006,...,M,SIRES,NJ08,D,202-225-7919,RepSires,22510,https://sires.house.gov/contact-me/email-me,https://sires.house.gov,https://youtube.com/RepSiresNJ13
8,P000096,1937-01-25,N00000751,,2370 Rayburn House Office Building,0,pascrell,202-225-5782,H6NJ08118,1996,...,M,PASCRELL,NJ09,D,202-225-5751,BillPascrell,478,https://pascrell.house.gov/contact/email-me,http://pascrell.house.gov,https://youtube.com/RepPascrell
9,P000604,1958-12-17,N00034639,,132 Cannon House Office Building,0,DonaldPayneJr,202-225-4160,H2NJ10154,2012,...,M,PAYNE,NJ10,D,202-225-3436,RepDonaldPayne,90668,http://payne.house.gov/contact/email-me,http://payne.house.gov,


## Once we have all these data sets, we can combine them
### We use the "cid" field that they share
I have been careful when creating the data sets so that this part is made very easy.
We only have to use the *merge* method of pandas, as follows:

In [67]:
#merge example (we can merge dfCandidates with any of the 3 others data sets)
pd.merge(dfContrib, dfCandidates)

,cid,indivs,org_name,pacs,total,year,bioguide_id,birthdate,comments,congress_office,...,gender,lastname,office,party,phone,twitter_id,votesmart_id,webform,website,youtube_url
0,N00036154,30400,"Conner, Strong & Buckelew",0,30400,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
1,N00036154,1000,Plumbers/Pipefitters Union,25000,26000,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
2,N00036154,21800,Charter School Management,0,21800,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
3,N00036154,1150,Intl Brotherhood of Electrical Workers,20000,21150,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
4,N00036154,1000,Carpenters & Joiners Union,20000,21000,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
5,N00036154,20950,Cooper University Hospital,0,20950,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
6,N00036154,0,American Federation of Teachers,20000,20000,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
7,N00036154,0,Ironworkers Union,20000,20000,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
8,N00036154,0,Laborers Union,20000,20000,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,
9,N00036154,0,Painters & Allied Trades Union,20000,20000,2014,N000188,1958-12-13,,1531 Longworth House Office Building,...,M,NORCROSS,NJ01,D,202-225-6501,DonaldNorcross,,http://norcross.house.gov/contact,https://norcross.house.gov,


In [71]:
## is the candidate summary useful?
#the method is "candSummary", I didn't make the data frame yet as it doen't share the same structure as the one before...
#I will make the data frame if it is needed


#sketch:

# r = requests.get('https://www.opensecrets.org/api/?method=candSummary&cid=N00007360&cycle=2016&apikey=925fb8825456ce94554138f1945e41de&output=json')
# print(r.text)
# j=r.json()
# j["response"]